Process a folder full of MassBank.txt files

In [13]:
import glob
import re
import string
import shutil
import os

In [2]:
# Merges two dictionaries where key - mass as string, value = inten as int
# This is very simplistic - values for the same key are simply added - could be much more sophisticated
# i.e. check for peaks with wrong defect, dimers, isotopes, etc.
def merge_my_dicts(dict1, dict2):
    for k, v in dict2.iteritems():
        if k in dict1.keys():
            dict1[k] = dict1[k] + v
        else:
            dict1[k] = v

In [3]:
# Reads a MassBank file.
# The header lines are read without processing and converted to a list - headers
# the peaks are kept as strings: this means the values from different files will be the same
# without having to worry about rounding errors; the assumption is that file from the same compound
# were processed at the same time with different params, so the values *should* be the same...
def read_massbank_file(f_name):
    headers = []
    peaks={}
    processing_peaks = False
    mw=0
    with open(f_name) as f:
        for line in f:
            if '//' in line:
                break
            elif processing_peaks:        
                parts = line.strip().split(' ')
                mass_as_str, inten_as_str = parts[0], parts[1]
                peaks[mass_as_str] = int(inten_as_str)
            elif 'PK$' not in line:     #nothing to do with peaks...
                headers += [line[:-1]]  #remove the newline character
                if 'CH$EXACT_MASS' in line:
                    parts=line.split(':')
                    mw = float(parts[1])
            elif 'PK$PEAK:' in line:    #start of peak secion
                processing_peaks = True
            
    return headers, peaks, mw

In [4]:
# Writes a MassBank format file from a dictionary with key = mass as string, value = inten as int
# In order to threshold and sort properly the mass strings are converted to floats
# The intensity is normalized so the base peak is 10000 and the relative inten is one tenth of that
# but at least 1
# We don't write any massed > upper_limit
def write_massbank_file(name, headers, peaks, tolerance, thresholdPercent=0.0, upper_limit=10000):
    #convert to float and filter values greater than the upper mass limit
    to_write = sorted([(float(k), v) for k, v in peaks.iteritems()])
    #print len(to_write), 'peaks initially'
    last_mass, last_inten = to_write[0]
    # centroid peaks that are closer than tolerance - can't assume there are only two
    centroided = []
    for mass, inten in to_write[1:]:
        if mass - last_mass < tolerance:
            temp = last_mass*last_inten + mass*inten
            last_inten += inten
            last_mass = temp/last_inten
        else:
            centroided += [(last_mass, last_inten)]
            last_mass, last_inten = mass, inten
    centroided += [(last_mass, last_inten)]      #write the final values
    #print len(centroided), 'after centroiding'
    #print to_write
    #filter by mass; we do this before calculating the base peak and threshold
    to_write = [(mass, inten) for mass, inten in centroided if mass < upper_limit]
    #print len(to_write), '<', upper_limit
    base_inten=max([v for m, v in to_write])
    scale = 10000.0 / base_inten
    if base_inten == 0:
        print to_write
    #
    abs_threshold = base_inten * thresholdPercent / 100
    to_write = filter(lambda t: t[1] >= abs_threshold, to_write)
    #print len(to_write), 'after threshold'
    #print to_write
    with open(name + ".txt", 'w') as f:
        for h in headers:
            f.write(h)
        f.write("PK$NUM_PEAK:  {}\n".format(len(to_write)))
        f.write("PK$PEAK: m/z int. rel.int.\n")
        for t in sorted(to_write):
            rel_int = int( t[1]*scale/10)   #rel int is inten /10 but at least 1
            f.write("  {:.4f} {:.0f} {:.0f}\n".format(t[0], t[1]*scale, max([1,rel_int])))
        f.write('//')

Can add '/Test' to the next line for testing on a subset of files

In [9]:
cd /Users/ronbonner/Data/HMDB spectra/

/Users/ronbonner/Data/HMDB spectra


In [15]:
mkdir 'positive'

mkdir: positive: File exists


In [11]:
mkdir 'negative'

In [16]:
pwd

u'/Users/ronbonner/Data/HMDB spectra'

In [25]:
# Remove redundant library entries and separate positive from negative
#
# Process a folder of MassBank files as derived from an Analyst library
# This is specific to the pestcide libraries where each compound has several entries that are very similar
# the approach is to track the name and build a dictionary with key = mass as string, inten = int
# when a duplicate mass is encountered the values (intensities) are added; they are re-normalized later
# The header lines are derived from the first instance of a compound: asumption is that they are the same
# When a compound name changes the existing spectrum, if any, is written 
compounds = set()
last_name = ""
last_polarity = ''
header_lines = []
peaks = {}
percent_threshold = 0.0
upper_limit_slop = 4     #gets added to mw; if == 3 isotope peaks will be included
in_count, pos_count,neg_count = 0,0,0
#
for f_name in glob.glob('HMDB Massbank files/*.txt'):
    head, tail = os.path.split(f_name)  # get the last portion of the file name
    in_count += 1
    m = re.search('(.+)\, (.*)-.*\.txt', tail)
    compound_name = m.groups()[0]
    polarity_str = m.groups()[1]

    #
#    out_dir = ''
#    if 'positive' in polarity_str:
#        pos_count += 1
#        out_dir = 'positive/'
#    elif 'negative'in polarity_str:
#        out_dir = 'negative/'
#        neg_count += 1
#
    if last_name != compound_name or polarity_str != last_polarity:
        #print f_name, compound_name, polarity_str
        #print "{}, {}, {}".format(compound_name, polarity_str, out_dir)
        if len(header_lines) > 0:  #something to write
            #print "Writing file"
            out_file = last_polarity +'/' + last_name
            write_massbank_file(out_file, header_lines, peaks, 0.002, 
                                0, upper_limit=mw+upper_limit_slop)
            #write_massbank_file('out/' + last_name, header_lines, peaks, 0.01, thresholdPercent=0.1)
            if 'positive' in last_polarity:
                pos_count += 1
            elif 'negative'in last_polarity:
                neg_count += 1
                
        last_name = compound_name
        last_polarity = polarity_str
    else:
        print 'This:', compound_name, polarity_str, 'Last:', last_name, last_polarity
    peaks={}             #clear the peak list; headers are cleared in read_massbank_file
    header_lines, peaks_in, mw = read_massbank_file(f_name)
    merge_my_dicts(peaks, peaks_in)
else:
    if len(header_lines) > 0:  #something write
        out_file = last_polarity +'/' + last_name
        write_massbank_file(out_file, header_lines, peaks, 0.002, 
                                0, upper_limit=mw+upper_limit_slop)
        if 'positive' in last_polarity:
            pos_count += 1
        elif 'negative'in last_polarity:
            neg_count += 1

print 'Done.', in_count, 'read', pos_count, 'pos,', neg_count,'neg,'

This: 2-Methylhippuric acid positive Last: 2-Methylhippuric acid positive
This: 4-Methoxycinnamic acid negative Last: 4-Methoxycinnamic acid negative
This: 6-Methyladenine negative Last: 6-Methyladenine negative
This: Cotinine positive Last: Cotinine positive
This: Cytidine triphosphate positive Last: Cytidine triphosphate positive
This: Daidzein negative Last: Daidzein negative
This: Trimethylamine positive Last: Trimethylamine positive
Done. 867 read 400 pos, 460 neg,


In [ ]:
mkdir 'out_mol'

In [ ]:
# Extract MOL files
# Essentially the same as the routine that removes redundant libraries, but only copies one MOL file
# to the output directory
last_name = ""
in_num = 0
out_num= 0
#
for f_name in glob.glob('*.mol'):
    in_num += 1
    m = re.search('(.+)\,.*\.mol', f_name)
    compound_name = m.groups()[0]
    if last_name != compound_name:
        if len(last_name) > 0:       #something to write
            shutil.copyfile(f_name, 'out_mol/' + compound_name + '.mol')
            out_num += 1
        last_name = compound_name
else:
    if len(last_name) > 0:       #something to write
        shutil.copyfile(f_name, 'out_mol/' + compound_name + '.mol')
        out_num += 1
#
print "Done:", in_num, "read,", out_num, "written"